# Imports & Setup

In [17]:
import datetime as dt
import time
import random
import logging

from optibook.synchronous_client import Exchange
from libs import print_positions_and_pnl, round_down_to_tick, round_up_to_tick

from IPython.display import clear_output

from stock_sentiment_analysis import analyze_sentiment_and_stock

logging.getLogger('client').setLevel('ERROR')    

# Function definitions

In [18]:
def insert_quotes(exchange, instrument, bid_price, ask_price, bid_volume, ask_volume):
    if bid_volume > 0:
        # Insert new bid limit order on the market
        exchange.insert_order(
            instrument_id=instrument.instrument_id,
            price=bid_price,
            volume=bid_volume,
            side='bid',
            order_type='limit',
        )
        
        # Wait for some time to avoid breaching the exchange frequency limit
        time.sleep(0.05)

    if ask_volume > 0:
        # Insert new ask limit order on the market
        exchange.insert_order(
            instrument_id=instrument.instrument_id,
            price=ask_price,
            volume=ask_volume,
            side='ask',
            order_type='limit',
        )

        # Wait for some time to avoid breaching the exchange frequency limit
        time.sleep(0.05)

# Main algorithm

In [19]:
class StockInfo:
    def __init__(self, buy_price, sell_price, bid_price, ask_price, bid_volume, ask_volume):
        self.buy = buy_price
        self.sell = sell_price
        self.bid_price = bid_price
        self.ask_price = ask_price
        self.bid_volume = bid_volume
        self.ask_volume = ask_volume
        
stock_dict = {
    'NVDA': StockInfo(buy_price=0.0, sell_price=1000.0, bid_price=0, ask_price=0, bid_volume=0,  ask_volume=0),
    'ING': StockInfo(buy_price=0.0, sell_price=1000.0, bid_price=0, ask_price=0, bid_volume=0,  ask_volume=0),
    'SAN': StockInfo(buy_price=0.0, sell_price=1000.0, bid_price=0, ask_price=0, bid_volume=0,  ask_volume=0),
    'PFE': StockInfo(buy_price=0.0, sell_price=1000.0, bid_price=0, ask_price=0, bid_volume=0,  ask_volume=0),
    'CSCO': StockInfo(buy_price=0.0, sell_price=1000.0, bid_price=0, ask_price=0, bid_volume=0,  ask_volume=0),
    'NVDA_B': StockInfo(buy_price=0.0, sell_price=1000.0, bid_price=0, ask_price=0, bid_volume=0,  ask_volume=0),
    'ING_B': StockInfo(buy_price=0.0, sell_price=1000.0, bid_price=0, ask_price=0, bid_volume=0,  ask_volume=0),
    'SAN_B': StockInfo(buy_price=0.0, sell_price=1000.0, bid_price=0, ask_price=0, bid_volume=0,  ask_volume=0),
    'PFE_B': StockInfo(buy_price=0.0, sell_price=1000.0, bid_price=0, ask_price=0, bid_volume=0,  ask_volume=0),
    'CSCO_B': StockInfo(buy_price=0.0, sell_price=1000.0, bid_price=0, ask_price=0, bid_volume=0,  ask_volume=0),
}

mid_price_dict = {
    'NVDA': 0,
    'ING': 0,
    'SAN': 0,
    'PFE': 0,
    'CSCO': 0,
    'NVDA_B': 0,
    'ING_B': 0,
    'SAN_B': 0,
    'PFE_B': 0,
    'CSCO_B': 0
}

In [34]:
exchange = Exchange()
exchange.connect()

instruments = exchange.get_instruments()

QUOTED_VOLUME = 10
FIXED_MINIMUM_CREDIT = 0.15
PRICE_RETREAT_PER_LOT = 0.005
POSITION_LIMIT = 100

selected_instruments = ['NVDA', 'ING', 'SAN', 'PFE', 'CSCO']
instruments_list = {instrument.instrument_id: instrument for instrument in instruments.values() if instrument.instrument_id in selected_instruments}

detected_company = []
while True:
    print(f'')
    print(f'-----------------------------------------------------------------')
    print(f'TRADE LOOP ITERATION ENTERED AT {str(dt.datetime.now()):18s} UTC.')
    print(f'-----------------------------------------------------------------')

    # Display our own current positions in all stocks, and our PnL so far
    print_positions_and_pnl(exchange)
    
    social_feeds = exchange.poll_new_social_media_feeds()
    
    if not social_feeds:
        # print(f'{dt.datetime.now()}: no new messages')
        adjusted_output = 0.0
    else:
        for feed in social_feeds:
            text = feed.post
            # print(f'{feed.timestamp}: {feed.post}')
            detected_company, predicted_sentiment, sentiment_score, adjusted_output = analyze_sentiment_and_stock(text)
    
    for instrument in instruments_list.values():
        
        dual_instrument = instruments[instrument.instrument_id + '_B']
        
        # Remove all existing (still) outstanding limit orders
        exchange.delete_orders(instrument.instrument_id)
        exchange.delete_orders(dual_instrument.instrument_id)
    
        # Obtain order book and only skip this instrument if there are no bids or offers available at all on that instrument,
        # as we we want to use the mid price to determine our own quoted price
        instrument_order_book = exchange.get_last_price_book(instrument.instrument_id)
        dual_instrument_order_book = exchange.get_last_price_book(dual_instrument.instrument_id)
        if not (instrument_order_book and instrument_order_book.bids and instrument_order_book.asks):
            print(f'{instrument.instrument_id:>6s} --     INCOMPLETE ORDER BOOK')
            continue
        if not (dual_instrument_order_book and dual_instrument_order_book.bids and dual_instrument_order_book.asks):
            print(f'{dual_instrument.instrument_id:>6s} --     INCOMPLETE ORDER BOOK')
            continue
    
        # Obtain own current position in instrument
        position = exchange.get_positions()[instrument.instrument_id]
        dual_position = exchange.get_positions()[dual_instrument.instrument_id]

        # Obtain best bid and ask prices from order book to determine mid price
        best_bid_price = instrument_order_book.bids[0].price
        best_ask_price = instrument_order_book.asks[0].price
        mid_price = (best_bid_price + best_ask_price) / 2.0 
        
        dual_best_bid_price = dual_instrument_order_book.bids[0].price
        dual_best_ask_price = dual_instrument_order_book.asks[0].price
        dual_mid_price = (dual_best_bid_price + dual_best_ask_price) / 2.0 

        # Calculate bid and ask volumes to insert, taking into account the exchange position_limit
        max_volume_to_buy = POSITION_LIMIT - position
        max_volume_to_sell = POSITION_LIMIT + position
        
        dual_max_volume_to_buy = POSITION_LIMIT - dual_position
        dual_max_volume_to_sell = POSITION_LIMIT + dual_position
        
        # Detecte which instrument is influenced
        # Calculate our fair/theoretical price based on the market mid price and our current position
        if instrument.instrument_id == detected_company:
            theoretical_price = mid_price - PRICE_RETREAT_PER_LOT * position + adjusted_output
            dual_theoretical_price = dual_mid_price - PRICE_RETREAT_PER_LOT * dual_position + adjusted_output
        else: 
            theoretical_price = mid_price - PRICE_RETREAT_PER_LOT * position 
            dual_theoretical_price = dual_mid_price - PRICE_RETREAT_PER_LOT * dual_position 

        bid_price = round_down_to_tick(theoretical_price - FIXED_MINIMUM_CREDIT, instrument.tick_size)
        ask_price = round_up_to_tick(theoretical_price + FIXED_MINIMUM_CREDIT, instrument.tick_size)  
        
        dual_bid_price = round_down_to_tick(dual_theoretical_price - FIXED_MINIMUM_CREDIT, dual_instrument.tick_size)
        dual_ask_price = round_up_to_tick(dual_theoretical_price + FIXED_MINIMUM_CREDIT, dual_instrument.tick_size)
        
        # Calculate final bid and ask prices to insert
        bid_price = min(stock_dict[instrument.instrument_id].sell, bid_price)
        ask_price = max(stock_dict[instrument.instrument_id].buy, ask_price)
        
        dual_bid_price = min(stock_dict[dual_instrument.instrument_id].sell, dual_bid_price)
        dual_ask_price = max(stock_dict[dual_instrument.instrument_id].buy, dual_ask_price)
        
        stock_dict[instrument.instrument_id].bid_price = bid_price
        stock_dict[instrument.instrument_id].ask_price = ask_price
        stock_dict[dual_instrument.instrument_id].bid_price = dual_bid_price
        stock_dict[dual_instrument.instrument_id].ask_price = dual_ask_price        
        
        if mid_price > dual_mid_price: #sell A, buy B
            bid_volume = 0
            ask_volume = min(QUOTED_VOLUME, max_volume_to_sell)
                
            dual_bid_volume = min(QUOTED_VOLUME, dual_max_volume_to_buy)
            dual_ask_volume = 0

        
        elif mid_price < dual_mid_price: #buy A, sell B
            bid_volume = min(QUOTED_VOLUME, max_volume_to_buy)
            ask_volume = 0
                
            dual_bid_volume = 0
            dual_ask_volume = min(QUOTED_VOLUME, dual_max_volume_to_sell)
            
        else:
            bid_volume = 0
            ask_volume = 0
                
            dual_bid_volume = 0
            dual_ask_volume = 0


        if instrument.instrument_id == detected_company:
            if adjusted_output > 0:
                bid_volume = bid_volume + 5 + min(QUOTED_VOLUME, max_volume_to_buy)
                dual_bid_volume = dual_bid_volume + 5 + min(QUOTED_VOLUME, dual_max_volume_to_buy)
            elif adjusted_output < 0:
                ask_volume = ask_volume - 5 + min(QUOTED_VOLUME, max_volume_to_sell)
                dual_ask_volume = dual_ask_volume - 5 + min(QUOTED_VOLUME, dual_max_volume_to_sell)
            else: continue
            
        stock_dict[instrument.instrument_id].bid_volume = bid_volume
        stock_dict[instrument.instrument_id].ask_volume = ask_volume
        stock_dict[dual_instrument.instrument_id].bid_volume = dual_bid_volume
        stock_dict[dual_instrument.instrument_id].ask_volume = dual_ask_volume
        
        trade_history = exchange.get_trade_history(instrument.instrument_id)[-1:]
        dual_trade_history = exchange.get_trade_history(dual_instrument.instrument_id)[-1:]
        
        if trade_history:
            # print(trade_history[0])
            if trade_history[0].side == 'bid':
                stock_dict[instrument.instrument_id].buy = trade_history[0].price
            else:
                stock_dict[instrument.instrument_id].sell = trade_history[0].price
                
        if dual_trade_history:
            # print(trade_history[0])
            if dual_trade_history[0].side == 'bid':
                stock_dict[dual_instrument.instrument_id].buy = dual_trade_history[0].price
            else:
                stock_dict[dual_instrument.instrument_id].sell = dual_trade_history[0].price

    for instrument in instruments.values():
        key = instrument.instrument_id
        
        # Remove all existing (still) outstanding limit orders
        exchange.delete_orders(instrument.instrument_id)
        
        stock = stock_dict[key]
        
        bid_price = stock.bid_price
        ask_price = stock.ask_price
        bid_volume = stock.bid_volume
        ask_volume = stock.ask_volume
        
        # Insert new quotes
        insert_quotes(exchange, instrument, bid_price, ask_price, bid_volume, ask_volume)
        
        
    detected_company = []
    adjusted_output = 0.0
    # Wait for a few seconds to refresh the quotes
    print(f'\nWaiting for one second.')
    time.sleep(1)
    
    # Clear the displayed information after waiting
    clear_output(wait=True)


-----------------------------------------------------------------
TRADE LOOP ITERATION ENTERED AT 2023-11-19 00:22:25.207801 UTC.
-----------------------------------------------------------------
Positions:
  NVDA      :   -7
  ING       :   -6
  SAN       :   12
  PFE       :  -24
  NVDA_B    :    2
  CSCO      :  -11
  ING_B     :   16
  PFE_B     :   26
  SAN_B     :   36
  CSCO_B    :  -37

PnL: 2795.78


KeyboardInterrupt: 

In [11]:
# # 平仓

# MIN_SELLING_PRICE = 0.10
# MAX_BUYING_PRICE = 100000.00

# positions = exchange.get_positions()
# pnl = exchange.get_pnl()

# print(f'Positions before: {positions}')
# print(f'\nPnL before: {pnl:.2f}')

# print(f'\nTrading out of positions')
# for iid, pos in positions.items():
#     if pos > 0:
#         print(f'-- Inserting sell order for {pos} lots of {iid}, with limit price {MIN_SELLING_PRICE:.2f}')
#         exchange.insert_order(iid, price=MIN_SELLING_PRICE, volume=pos, side='ask', order_type='ioc')
#     elif pos < 0:
#         print(f'-- Inserting buy order for {abs(pos)} lots of {iid}, with limit price {MAX_BUYING_PRICE:.2f}')
#         exchange.insert_order(iid, price=MAX_BUYING_PRICE, volume=-pos, side='bid', order_type='ioc')
#     else:
#         print(f'-- No initial position in {iid}, skipping..')
    
#     time.sleep(0.10)

# time.sleep(1.0)

# positions = exchange.get_positions()
# pnl = exchange.get_pnl()
# print(f'\nPositions after: {positions}')
# print(f'\nPnL after: {pnl:.2f}')